In [1]:
import pandas as pd
import polars as pl
import numpy as np
import xgboost as xgb

starting_index = 133
ending_index = 2373
schema_overrides = {
    f'num_feature_{i}':pl.Float32 for i in range(starting_index, ending_index + 1, 1)
}
schema_overrides['customer_id'] = pl.Int32

main_overrides = {}
for i in range(1, starting_index, 1):
    main_overrides[f'num_feature_{i}'] = pl.Float32

for i in range(1, 200 - starting_index + 2, 1):
    main_overrides[f'cat_feature_{i}'] = pl.Int8

main_overrides['customer_id'] = pl.Int32
df_train_main = pl.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/train_main_features (1).parquet')
df_train_extra = pl.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/train_extra_features.parquet')
df_target = pl.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/train_target.parquet')
df_train_extra.head(5)

customer_id,num_feature_133,num_feature_134,num_feature_135,num_feature_136,num_feature_137,num_feature_138,num_feature_139,num_feature_140,num_feature_141,num_feature_142,num_feature_143,num_feature_144,num_feature_145,num_feature_146,num_feature_147,num_feature_148,num_feature_149,num_feature_150,num_feature_151,num_feature_152,num_feature_153,num_feature_154,num_feature_155,num_feature_156,num_feature_157,num_feature_158,num_feature_159,num_feature_160,num_feature_161,num_feature_162,num_feature_163,num_feature_164,num_feature_165,num_feature_166,num_feature_167,num_feature_168,…,num_feature_2337,num_feature_2338,num_feature_2339,num_feature_2340,num_feature_2341,num_feature_2342,num_feature_2343,num_feature_2344,num_feature_2345,num_feature_2346,num_feature_2347,num_feature_2348,num_feature_2349,num_feature_2350,num_feature_2351,num_feature_2352,num_feature_2353,num_feature_2354,num_feature_2355,num_feature_2356,num_feature_2357,num_feature_2358,num_feature_2359,num_feature_2360,num_feature_2361,num_feature_2362,num_feature_2363,num_feature_2364,num_feature_2365,num_feature_2366,num_feature_2367,num_feature_2368,num_feature_2369,num_feature_2370,num_feature_2371,num_feature_2372,num_feature_2373
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1000001,-0.116197,-0.009981,null,null,null,null,-0.067005,null,null,-0.001311,0.0,null,-0.087608,-0.011702,null,null,null,null,null,null,null,null,null,-0.024235,null,-0.049388,-0.034772,null,-0.038661,-0.075484,1.116927,-0.032538,-0.122623,-0.01873,-0.081526,null,…,-0.039189,null,0.0,null,null,-0.14431,null,null,-0.053982,null,0.0,-0.072911,null,-0.581945,null,-0.080336,0.440102,null,null,-0.020938,-0.142138,null,-0.023246,0.0,null,-0.020769,-0.14374,null,-0.00276,null,0.0,-0.030607,-0.015407,-0.404816,-0.395589,null,0.0
1000002,-0.116197,-0.009981,null,null,null,null,-0.067005,-0.088452,null,0.000073,0.0,null,-0.087608,-0.011702,-0.280531,null,-0.001478,null,null,null,-0.15721,null,null,-0.024235,0.550582,-0.049388,-0.034772,null,-0.038661,-0.075484,-0.232695,-0.032538,-0.122623,0.176785,-0.081526,null,…,-0.023996,null,0.0,null,null,-0.14431,null,null,-0.053982,2.186477,0.0,-0.072911,null,0.04981,-0.035824,-0.080336,0.186329,null,null,-0.020938,-0.122687,2.700091,-0.023246,0.0,null,-0.020769,-0.14374,null,-0.001345,null,0.0,-0.030607,-0.047964,1.630249,-0.395589,-0.201813,0.0
1000003,-0.116197,-0.009981,null,null,null,null,-0.067005,-0.088452,null,-0.001343,0.0,null,-0.087608,-0.011702,3.639459,null,-0.001488,null,null,null,null,null,null,-0.024235,-0.325277,-0.049388,-0.034772,null,null,-0.075484,-0.232695,-0.032538,-0.122623,-0.01873,-0.081526,null,…,-0.039189,null,0.0,null,null,-0.14431,null,null,-0.053982,-0.140279,0.0,-0.072911,null,-0.660424,-0.035824,-0.080336,-0.942263,null,null,null,-0.228213,-0.143084,-0.023246,0.0,null,-0.020769,-0.14374,null,-0.00276,null,0.0,-0.030607,-0.045637,-0.404816,-0.395589,-0.271623,0.0
1000004,-0.116197,-0.009981,null,null,null,null,-0.067005,null,null,-0.001272,0.0,-1.18569,-0.087608,-0.011702,1.679464,null,null,-0.898284,null,null,null,null,null,-0.024235,null,-0.049388,-0.034772,null,null,-0.075484,-0.232695,null,-0.122623,null,-0.081526,null,…,null,null,0.0,-0.318906,null,-0.14431,null,null,-0.053982,null,0.0,-0.072911,null,-0.683968,null,-0.080336,0.528574,-0.657523,null,null,-0.352937,null,-0.023246,0.0,0.0,-0.020769,-0.14374,null,-0.00276,null,0.0,-0.030607,-0.058314,-0.404816,null,null,0.0
1000005,null,null,null,null,null,null,null,-0.088452,null,null,0.0,null,null,-0.011702,0.046135,null,-0.001492,null,null,null,null,null,null,-0.024235,-0.150105,null,null,null,null,-0.075484,null,-0.032538,null,-0.01873,null,null,…,-0.039189,null,0.0,null,null,null,null,null,null,-0.140279,0.0,-0.072911,nu

In [2]:
df_train_extra = df_train_extra.with_columns([
    pl.col(col).cast(pl.Float32) for col in df_train_extra.columns
]).to_pandas()
df_train_main = df_train_main.to_pandas()
df_train_extra = pd.merge(df_train_main, df_train_extra, on='customer_id', how='left')

In [3]:
cat_features, num_features = [], []
for col in df_train_extra.columns:
    if 'cat' in col:
        cat_features.append(col)
    if 'num' in col:
        num_features.append(col)

len(cat_features), len(num_features)

(67, 2373)

In [4]:
df_target = df_target.to_pandas().drop(columns=['customer_id'])

In [5]:
df_train_extra.drop(columns=['customer_id'], inplace=True)

In [6]:
del df_train_main

In [7]:
params = {'objective': 'binary:logistic',
          'multi_strategy': 'one_output_per_tree',
'n_estimators': 118,
    'learning_rate': 0.03883858415363916,
    'max_depth': 8,
    'reg_alpha': 0.49408561562877995,
    'reg_lambda': 0.4416018461606211,
    'min_child_weight': 5,
    'subsample': 0.8159475144145236,
    'colsample_bytree': 0.5591605629773047,
        'device': 'cuda',
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'early_stopping_rounds': 50,
        'random_state': 69,
    }

In [ ]:
col_index = 0
import torch
import gc
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# Подготовка данных (те же разбиения)
n_labels = 41
def macro_roc_auc(preds, dtrain):
    labels = dtrain.get_label().reshape(-1, n_labels)
    aucs = []
    for i in range(n_labels):
        if len(np.unique(labels[:, i])) == 2:
            aucs.append(roc_auc_score(labels[:, i], preds[:, i]))
        else:
            aucs.append(0.5)   # только один класс – нейтральное значение
    macro_auc = np.mean(aucs)
    # Возвращаем (имя_метрики, значение, higher_is_better) – третий элемент необязателен,
    # но для ясности оставляем. XGBoost учитывает его, если он есть.
    return 'macro_auc', -macro_auc
# Создаём DMatrix – формат данных для xgb.train
dtrain = xgb.DMatrix(df_train_extra.iloc[:650000, :], label=df_target.iloc[:650000, :].values)
dvalid = xgb.DMatrix(df_train_extra.iloc[650000:, :], label=df_target.iloc[650000:, :].values)

del df_train_extra
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 60, 240),
        'objective': 'binary:logistic',
        'tree_method': 'hist',
        'device': 'cuda',
        'random_state': 69,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }

    booster = xgb.train(
        param,
        dtrain,
        num_boost_round=1000,
        evals=[(dvalid, 'valid')],
        early_stopping_rounds=10,
        custom_metric=macro_roc_auc,
        verbose_eval=5
    )
    gc.collect()
    torch.cuda.empty_cache()
    score = -booster.best_score
    del booster
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True, timeout=14400)

print("Лучшее значение logloss:", study.best_value)
print("Лучшие параметры:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

# Для финальной модели можно обучить booster с лучшими параметрами
best_params = study.best_params
best_params['random state'] = 69
final_booster = xgb.train(
    best_params,
    dtrain,
    num_boost_round=param['n_estimators'],
    evals=[(dvalid, 'valid')],
    early_stopping_rounds=10,
    verbose_eval=5
)

# model = xgb.XGBClassifier(**params)
# model.fit(df_train_extra.iloc[:650000, :], df_target.iloc[:650000, :], 
#           eval_set=[(df_train_extra.iloc[650000:, :], df_target.iloc[650000:, :])],
#           verbose=5)

[I 2026-02-19 21:30:11,193] A new study created in memory with name: no-name-715a5b08-b076-48a6-9033-50027129b667


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [21:30:11] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	valid-logloss:0.10314	valid-macro_auc:-0.77547
[5]	valid-logloss:0.09680	valid-macro_auc:-0.80929
[10]	valid-logloss:0.09311	valid-macro_auc:-0.81515
[15]	valid-logloss:0.09061	valid-macro_auc:-0.81931
[20]	valid-logloss:0.08877	valid-macro_auc:-0.82123
[25]	valid-logloss:0.08737	valid-macro_auc:-0.82322
[30]	valid-logloss:0.08628	valid-macro_auc:-0.82466
[35]	valid-logloss:0.08541	valid-macro_auc:-0.82621
[40]	valid-logloss:0.08474	valid-macro_auc:-0.82769
[45]	valid-logloss:0.08419	valid-macro_auc:-0.82923
[50]	valid-logloss:0.08374	valid-macro_auc:-0.83051
[55]	valid-logloss:0.08337	valid-macro_auc:-0.83173
[60]	valid-logloss:0.08307	valid-macro_auc:-0.83273
[65]	valid-logloss:0.08281	valid-macro_auc:-0.83338
[70]	valid-logloss:0.08260	valid-macro_auc:-0.83402
[75]	valid-logloss:0.08241	valid-macro_auc:-0.83459
[80]	valid-logloss:0.08226	valid-macro_auc:-0.83508
[85]	valid-logloss:0.08212	valid-macro_auc:-0.83555
[90]	valid-logloss:0.08201	valid-macro_auc:-0.83608
[95]	valid-log

In [ ]:
# from sklearn.metrics import roc_auc_score
# preds = model.predict_proba(df_train_extra.iloc[650000:, :])
# print(roc_auc_score(df_target.iloc[650000:, :], preds, average='macro'))

In [ ]:
df_test_main = pl.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/test_main_features.parquet')
df_test_extra = pl.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/test_extra_features.parquet')


In [ ]:
df_test_extra = df_test_extra.with_columns([
    pl.col(col).cast(pl.Float32) for col in df_test_extra.columns
]).to_pandas()
df_test_main = df_test_main.to_pandas()
df_test_extra = pd.merge(df_test_main, df_test_extra, on='customer_id', how='left')

In [ ]:
df_test_extra.drop(columns=['customer_id'], inplace=True)
sample_submit = pd.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/sample_submit.parquet')

In [ ]:
# output = model.predict_proba(df_test_extra)
output = final_booster.predict(xgb.DMatrix(df_test_extra))

In [ ]:
predict_columns = [col for col in list(sample_submit.columns) if 'predict' in col]
submit = pd.DataFrame(output, columns=predict_columns)

In [ ]:
submit = submit.astype('float64')
submit['customer_id'] = sample_submit['customer_id']
submit.head(5)

In [ ]:
submit.to_parquet('submission.parquet', index=False)

In [ ]:
from joblib import dump, load
target_columns = list(df_target.columns)
dump(model, f'model_{target_columns[col_index]}.joblib')

In [ ]:
del df_train_extra

In [ ]:
df_test_main = pl.read_parquet('/kaggle/input/datasets/kirillpale228777/datafusionligalasosa/test_main_features.parquet')
